**Downloading Data Set**

In [1]:
import kagglehub

path = kagglehub.dataset_download("muhammadhananasghar/human-emotions-datasethes")

ModuleNotFoundError: No module named 'kagglehub'

**Making Train and Test Directory(Location)**

In [ ]:
train_directory = "/kaggle/input/human-emotions-datasethes/Emotions Dataset/Emotions Dataset/train"#Change path to train path
test_directory = "/kaggle/input/human-emotions-datasethes/Emotions Dataset/Emotions Dataset/test"#Change path to test path

CONFIGURATION = {
    "BATCH_SIZE": 16,
    "IM_SIZE": 256,
    "LEARNING_RATE": 1e-3,
    "N_EPOCHS": 20,
    "N_FILTERS": 6,
    "KERNEL_SIZE": 3,
    "N_STRIDES": 1,
    "POOL_SIZE": 2,
    "N_DENSE_1": 1024,
    "N_DENSE_2": 128,
    "NUM_CLASSES": 3,
    "PATCH_SIZE": 16,
    "PROJ_DIM": 768,
    "CLASS_NAMES": ["angry", "happy", "sad"],
}

**Making Train and Test Data Set**

In [ ]:
import tensorflow as tf

train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    train_directory,
    labels='inferred', #labels will be genrated as there stored folder name
    label_mode='categorical', #it work as one-hot-incoder like in case of angry label it will give angry value of 1 and happy-sad will get 0, such to all
    class_names=CONFIGURATION["CLASS_NAMES"],#put all labels(folder name)
    color_mode='rgb',
    batch_size=CONFIGURATION["BATCH_SIZE"], # Use updated batch size from CONFIGURATION
    image_size=(256, 256),
    shuffle=True,
    seed=42,
    validation_split=None,#if we dont have saperate test then put split value ie 0.2
    subset=None#if have split then need to specify it ie "traing" or "validation"(test)
)

In [ ]:
test_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    test_directory,
    labels='inferred',
    label_mode='categorical',
    class_names=CONFIGURATION["CLASS_NAMES"],
    color_mode='rgb',
    batch_size=CONFIGURATION["BATCH_SIZE"], # Use updated batch size from CONFIGURATION
    image_size=(256, 256),
    shuffle=True,
    seed=42
)

In [ ]:
#lets see images tensors
for i in train_dataset.take(1):
  print(i)

In [ ]:
#lets see the image
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 12))
for images, labels in train_dataset.take(1):
  for i in  range(16):
    plt.subplot(4,4,i+1)
    plt.imshow(images[i]/255.0)
    plt.title(tf.argmax(labels[i],axis=0).numpy())
    plt.axis("off")

In [ ]:
# Prefetch to improve performance by preparing the next batch while the current one is being processed
training_dataset = train_dataset.prefetch(tf.data.AUTOTUNE)

# Same for testing/validation dataset to speed up evaluation
testing_dataset = test_dataset.prefetch(tf.data.AUTOTUNE)




> **Making Model**



In [ ]:

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,InputLayer,Rescaling, Resizing

from tensorflow.keras.layers import Conv2D #Conv2D detects patterns like edges and shapes in images automatically for better training
from tensorflow.keras.layers import Flatten #Flatten reshapes the output of Conv2D/MaxPool2D layers 1D which is needed for Dense
from tensorflow.keras.layers import BatchNormalization #faster and more stable training|||PUT AFTER ALL Conv2D and Dense layer
from tensorflow.keras.layers import MaxPool2D #MaxPool2D reduces the spatial dimensions of feature maps while keeping the most important features.(less computing)


#Lets create resize_rescale_layer for input of image
resize_rescale_layer = Sequential([
    Resizing(256, 256),   # Resize all input images to 256x256 pixels for consistent input shape
    Rescaling(1./255)     # Normalize pixel values from [0,255] to [0,1] for faster and stable training
])


In [ ]:
emotion_model = tf.keras.Sequential(
    [
    InputLayer(input_shape = (None,None, 3), ),

    resize_rescale_layer,

    Conv2D(filters = CONFIGURATION["N_FILTERS"] , kernel_size = CONFIGURATION["KERNEL_SIZE"], strides = CONFIGURATION["N_STRIDES"] , padding='same',
    activation = 'relu'),

    BatchNormalization(),

    MaxPool2D (pool_size = CONFIGURATION["POOL_SIZE"], strides= CONFIGURATION["N_STRIDES"]*2),

    Conv2D(filters = CONFIGURATION["N_FILTERS"]*2 + 4, kernel_size = CONFIGURATION["KERNEL_SIZE"], strides=CONFIGURATION["N_STRIDES"], padding='same',
          activation = 'relu'),

    BatchNormalization(),

    MaxPool2D (pool_size = CONFIGURATION["POOL_SIZE"], strides= CONFIGURATION["N_STRIDES"]*2),

    Flatten(),

    Dense( CONFIGURATION["N_DENSE_1"], activation = "relu"),

    BatchNormalization(),

    Dense( CONFIGURATION['N_DENSE_2'], activation = "relu"),

    BatchNormalization(),

    Dense(CONFIGURATION["NUM_CLASSES"], activation = "softmax"),

])

emotion_model.summary()

**Training**

In [ ]:

from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import CategoricalCrossentropy #CategoricalCrossentropy() tells the model how wrong its predicted class probabilities are, and the optimizer uses that to adjust weights during training.

from tensorflow.keras.metrics import CategoricalAccuracy, TopKCategoricalAccuracy #Checks if the model’s top (highest) predicted class matches the true class.It’s “Top-1” accuracy — only counts as correct if the model’s #1 guess is right

from tensorflow.keras.metrics import TopKCategoricalAccuracy #Checks if the true class is among the top-k predicted classes.For example, if k=2, it’s correct if the true class is in the top 2 probabilities.



loss_function = CategoricalCrossentropy()

metrics = [CategoricalAccuracy (name = "accuracy"), TopKCategoricalAccuracy( k=2 , name="top_k_accuracy")]

In [ ]:

#main training
emotion_model.compile(
    optimizer = Adam(learning_rate=CONFIGURATION["LEARNING_RATE"]),
    loss = loss_function,
    metrics = metrics
)

In [ ]:

History = emotion_model.fit(
training_dataset,#Each batch gives (X_train, y_train) — images + correct emotion labels.
validation_data = testing_dataset,#Used to check how well the model generalizes to unseen data after each epoch
epochs = 20, #Model sees every training image 10 times
verbose = 1,
)

In [ ]:
emotion_model.save("emotion_model.h5")

In [ ]:
#/////////////////UNCOMMENT TO DOWNLOAD MODEL///////////////////////



# from google.colab import files

# files.download('emotion_model.h5')

***Please refer Model-Test.ipynb to test the Model***